In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [5]:
scaler = MinMaxScaler()
scaler.feature_range = (0, 1)
scale_factor = 1/0.00041967

In [ ]:
heikin_ashi_data=pd.read_csv("dataset.csv")

print("Close Price Visualization")
plt.plot(heikin_ashi_data.Close)

In [ ]:
data_train, data_test = train_test_split(heikin_ashi_data, test_size=0.30, shuffle=False)

print(data_train.shape)
print(data_test.shape)

In [8]:
train_close_data = data_train.iloc[:, 4:5].values
test_close_data = data_test.iloc[:, 4:5].values

In [9]:
data_train_scaled = scaler.fit_transform(train_close_data)

In [10]:
import numpy as np
x_train = []
y_train = [] 

for i in range(100, data_train_scaled.shape[0]):
    x_train.append(data_train_scaled[i-100: i])
    y_train.append(data_train_scaled[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train) 

In [ ]:
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))


model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))


model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))


model.add(LSTM(units = 50))
model.add(Dropout(0.2))


model.add(Dense(units = 1))

In [ ]:
import tensorflow as tf
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train,epochs = 32)

In [13]:
model.save('keras_model.keras')

In [14]:
past_data = pd.DataFrame(train_close_data[-242:])
test_df = pd.DataFrame(test_close_data)
final_df = pd.concat([past_data, test_df], ignore_index=True)
input_data = scaler.fit_transform(final_df)

In [15]:
x_test = []
y_test = []
for i in range(100, input_data.shape[0]):
   x_test.append(input_data[i-100: i])
   y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
y_pred = model.predict(x_test)

In [17]:
y_pred = y_pred * scale_factor
y_test = y_test * scale_factor

In [ ]:
loss= mean_absolute_error(y_test, y_pred)
loss_percentage = (loss / np.mean(y_test)) * 100
print("Mean absolute error on test set: {:.2f}%".format(loss_percentage))

In [ ]:
plt.figure(figsize = (12,6))
plt.plot(y_test, 'g', label = "Original Stock Price")
plt.plot(y_pred, 'y', label = "Predicted Stock Price")
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()